In [ ]:
!pip install streamlit langchain OpenAI 

In [ ]:
# Bring in deps
import os 
from apikey import apikey 
import datetime
import json

 
from langchain.chat_models import ChatOpenAI

#from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import PALChain
from langchain.agents import initialize_agent, Tool      
os.environ['OPENAI_API_KEY'] = apikey

In [ ]:
from calpy import Calendar, Event

No events on the specified date.
Events on 2023-05-18:
- Wank-a-thon at 23:00:00
- Wank-a-thon at 23:00:00
Events on 2023-05-18:
- Wank-a-thon at 23:00:00
- Wank-a-thon at 23:00:00
Events on 2023-05-19:
- example at 23:00:00
- example at 23:00:00


In [ ]:
calendar = Calendar().from_json('events.json')


In [ ]:
#Get todays date
today = datetime.datetime.now()
def todayDate():
	return today.strftime('%m/%d/%y')


In [ ]:

 

# functions for scheduling:

# schedule available time

llm = OpenAI(temperature=0.9, model_name='gpt-3.5-turbo') 

pal_chain = PALChain.from_math_prompt(llm, verbose=True)

tools = [
	Tool(
		name = "today's date",
		func = lambda string: todayDate(),
		description="use to get today's date",
		),
	Tool(
		name = 'check events',
		func = lambda string: calendar.get_events_on_date(string),
		description="Use to check on what events are coming up and needs to taken into account. The input to this tool should be a string in this format mm/dd/yy. This is the only way for you to answer questions about upcoming events. This tool will reply with planned events for the specified date in 24hour time, for example: 15:00 and 3pm are the same.",
		),
	Tool(
		name = 'schedule event',
		func = lambda string: calendar.add_event(string),
		description="Use to schedule an event for a given date and time. The input to this tool should be a comma separated list of 2 strings: date and time in format: mm/dd/yy, hh:mm, convert date and time to these formats. For example, `12/31/23, 10:00` would be the input if for Dec 31'st 2023 at 10am",
		),
	Tool(
        name = "PAL",
        func = pal_chain.run,
        description = "useful for when you need to answer questions about math or word problems or date comparisons"),
]
memory = ConversationBufferMemory(memory_key="chat_history")

agent_chain = initialize_agent(tools, llm, agent='zero-shot-react-description', memory=memory, verbose=True)


# Show stuff to the screen if there's a prompt
#if #prompt: 
   # title = title_chain.run(prompt)
   # wiki_research = wiki.run(prompt) 
   # script = script_chain.run(title=title, wikipedia_research=wiki_research)



/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
agent_chain.run(["i have to take my medicine every wednesday from the 18/05/2023 at 12:00 PM and onwards, schedule it please"])



> Entering new AgentExecutor chain...
 I need to schedule the event for my medicine
Action: schedule event
Action Input: 18/05/2023, 12:00
Observation: None
Thought: I will need to check if this event is already in the system
Action: check events
Action Input: 18/05/2023

TypeError: ignored